---
## Step 1 — GPU Check & Install Dependencies

In [ ]:
#@title 1a. Verify GPU is available
!nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv
print()
!nvidia-smi | grep 'CUDA Version'

In [ ]:
#@title 1b. Install required packages
!pip install -q onnxruntime-gpu>=1.16.0 onnx pycocotools>=2.0.7 \
    PyYAML>=6.0 matplotlib>=3.7.0 pandas>=2.0.0 opencv-python-headless>=4.8.0

# Verify GPU provider is available
import onnxruntime as ort
print(f"\n✅ ONNX Runtime {ort.__version__}")
print(f"   Providers: {ort.get_available_providers()}")
assert 'CUDAExecutionProvider' in ort.get_available_providers(), \
    "❌ CUDAExecutionProvider not found! Make sure you enabled GPU: Settings → Accelerator → GPU T4x2 or P100"

---
## Step 2 — Clone Your GitHub Repository

This pulls the evaluation code, utility scripts, and config. ONNX models and images come from Kaggle datasets.

In [ ]:
#@title 2. Clone the repo
import os

REPO_URL = "https://github.com/Sasankamadura/FYP-Final-Testing.git"
WORKSPACE = "/kaggle/working/FYP-Final-Testing"

if os.path.exists(WORKSPACE):
    print(f"Repo already cloned at {WORKSPACE}, pulling latest...")
    !cd "{WORKSPACE}" && git pull
else:
    !git clone "{REPO_URL}" "{WORKSPACE}"

os.chdir(WORKSPACE)
print(f"\n✅ Working directory: {os.getcwd()}")
print(f"   Contents: {os.listdir('.')}")

---
## Step 3 — Link Kaggle Datasets into Workspace

Kaggle datasets are mounted **read-only** under `/kaggle/input/`.
We create **symlinks** so the eval scripts find files at the expected paths.

### ⚠️ Update the dataset slugs below to match YOUR uploaded dataset names!

In [ ]:
#@title 3a. Configure Kaggle dataset paths
# ============================================================
# UPDATE THESE to match your Kaggle dataset slugs!
# Go to the "Input" panel on the right to see the exact paths.
# ============================================================

# Your Kaggle username (used in dataset paths)
KAGGLE_USER = "your-kaggle-username"  # <-- CHANGE THIS

# Dataset 1: ONNX checkpoints
CHECKPOINTS_DATASET = f"/kaggle/input/fyp-visdrone-checkpoints"

# Dataset 2: VisDrone images + annotations
VISDRONE_DATASET = f"/kaggle/input/fyp-visdrone-dataset"

print("Kaggle input datasets:")
print(f"  Checkpoints: {CHECKPOINTS_DATASET}")
print(f"  VisDrone:    {VISDRONE_DATASET}")

# Quick sanity check
for p in [CHECKPOINTS_DATASET, VISDRONE_DATASET]:
    if os.path.exists(p):
        print(f"  ✅ Found: {p}")
        print(f"     Contents: {os.listdir(p)[:10]}")
    else:
        print(f"  ❌ NOT FOUND: {p}")
        print(f"     → Check 'Add Data' panel on the right. Available inputs:")
        if os.path.exists('/kaggle/input'):
            print(f"       {os.listdir('/kaggle/input')}")

In [ ]:
#@title 3b. Create symlinks into workspace
import os
os.chdir(WORKSPACE)

# ---- Symlink checkpoint folders ----
links = {
    "Checkpoints - Baseline_Visdrone2019": os.path.join(CHECKPOINTS_DATASET, "Checkpoints - Baseline_Visdrone2019"),
    "Checkpoints - After improvements":    os.path.join(CHECKPOINTS_DATASET, "Checkpoints - After improvements"),
    "VisDrone Val image set":               os.path.join(VISDRONE_DATASET, "VisDrone Val image set"),
    "VisDrone test image set":              os.path.join(VISDRONE_DATASET, "VisDrone test image set"),
}

for link_name, target in links.items():
    link_path = os.path.join(WORKSPACE, link_name)
    
    # Remove existing link/dir if present
    if os.path.islink(link_path):
        os.unlink(link_path)
    elif os.path.isdir(link_path):
        import shutil
        shutil.rmtree(link_path)
    
    if os.path.exists(target):
        os.symlink(target, link_path)
        print(f"✅ {link_name} → {target}")
    else:
        print(f"❌ Target not found: {target}")
        print(f"   Available in dataset root: {os.listdir(os.path.dirname(target))}")

print("\n📁 Workspace contents:")
for item in sorted(os.listdir(WORKSPACE)):
    full = os.path.join(WORKSPACE, item)
    suffix = " → " + os.readlink(full) if os.path.islink(full) else ""
    print(f"  {'📁' if os.path.isdir(full) else '📄'} {item}{suffix}")

---
## Step 4 — Verify Workspace

Checks all directories, annotations, images, and model checkpoint files.

In [ ]:
#@title 4. Full workspace verification
import os, yaml
os.chdir(WORKSPACE)

# --- Check key directories ---
print("━" * 60)
print("  DIRECTORY CHECK")
print("━" * 60)
expected_dirs = [
    'eval',
    'eval/utils',
    'Checkpoints - Baseline_Visdrone2019',
    'Checkpoints - After improvements',
    'VisDrone Val image set',
    'VisDrone test image set/images',
]
for d in expected_dirs:
    exists = os.path.isdir(d)
    print(f"  {'✅' if exists else '❌'} {d}")

# --- Check annotations ---
ann = 'VisDrone Val image set/annotations_VisDrone_val.json'
print(f"\n  Annotations: {'✅' if os.path.exists(ann) else '❌'} {ann}")

# --- Count images ---
val_imgs = [f for f in os.listdir('VisDrone Val image set') if f.endswith(('.jpg','.png'))]
test_imgs = [f for f in os.listdir('VisDrone test image set/images') if f.endswith(('.jpg','.png'))]
print(f"  Val images: {len(val_imgs)}")
print(f"  Test images: {len(test_imgs)}")

# --- Load config & check each model ---
print(f"\n━" * 60)
print("  MODEL CHECKPOINT VERIFICATION")
print("━" * 60)
with open('eval/config.yaml') as f:
    config = yaml.safe_load(f)

all_ok = True
for key, mcfg in config['models'].items():
    exists = os.path.exists(mcfg['path'])
    status = '✅' if exists else '❌ MISSING'
    size_str = ""
    if exists:
        size_mb = os.path.getsize(mcfg['path']) / (1024*1024)
        size_str = f"({size_mb:.1f} MB)"
    else:
        all_ok = False
    print(f"  {status} {mcfg['name']:<40} {size_str}")

print()
if all_ok:
    print("✅ All models found! Ready to run evaluation.")
else:
    print("❌ Some models are missing. Check your Kaggle dataset paths in Step 3a.")

---
## Step 4b — Visual Sanity Check (Predictions vs Ground Truth)

Runs the **baseline model** on a random validation image and displays:
- **Left**: Ground-truth annotations (from COCO JSON)
- **Right**: Model predictions (from ONNX inference)

This verifies the **category mapping is correct** — class labels on both sides should match (e.g. a car should be labelled "car", not "van").

In [ ]:
#@title 4b. Visual comparison — Ground Truth vs Baseline Predictions
import os, sys, json, random
import numpy as np
import cv2
import yaml
from IPython.display import display, Image as IPImage

os.chdir(WORKSPACE)
sys.path.insert(0, os.path.join(WORKSPACE, "eval"))

from utils.onnx_inference import OnnxDetector
from utils.visualization import COLORS, CLASS_NAMES

# ── Load config & annotations ──────────────────────────────────
with open("eval/config.yaml") as f:
    config = yaml.safe_load(f)

ann_path = os.path.join(WORKSPACE, config["datasets"]["val"]["annotations"])
with open(ann_path) as f:
    coco_data = json.load(f)

cat_map = {c["id"]: c["name"] for c in coco_data["categories"]}
images_dir = os.path.join(WORKSPACE, config["datasets"]["val"]["images_dir"])

# ── Pick a random image that has enough annotations ─────────────
from collections import Counter
ann_per_img = Counter(a["image_id"] for a in coco_data["annotations"])
rich_images = [img for img in coco_data["images"] if ann_per_img.get(img["id"], 0) >= 10]
img_info = random.choice(rich_images)
img_path = os.path.join(images_dir, img_info["file_name"])
image = cv2.imread(img_path)
orig_h, orig_w = image.shape[:2]
print(f"Selected image: {img_info['file_name']} ({orig_w}x{orig_h}, "
      f"{ann_per_img[img_info['id']]} GT annotations)")

# ── Draw Ground-Truth ───────────────────────────────────────────
gt_img = image.copy()
gt_anns = [a for a in coco_data["annotations"] if a["image_id"] == img_info["id"]]
for ann in gt_anns:
    x, y, w, h = ann["bbox"]
    cid = ann["category_id"]
    idx = cid - 1
    color = COLORS[idx % len(COLORS)]
    name = cat_map.get(cid, f"cls{cid}")
    cv2.rectangle(gt_img, (int(x), int(y)), (int(x+w), int(y+h)), color, 2)
    label = f"GT: {name}"
    (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.45, 1)
    cv2.rectangle(gt_img, (int(x), int(y)-th-6), (int(x)+tw, int(y)), color, -1)
    cv2.putText(gt_img, label, (int(x), int(y)-4),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1)

# ── Run Baseline Model Inference ────────────────────────────────
baseline_cfg = config["models"]["baseline_rtdetr_r18"]
model_path = os.path.join(WORKSPACE, baseline_cfg["path"])
detector = OnnxDetector(model_path, input_size=(640, 640))

boxes, scores, class_ids = detector.predict(image, conf_threshold=0.3)
print(f"Baseline predictions (conf>0.3): {len(scores)} detections")
print(f"  Unique model labels: {sorted(set(int(c) for c in class_ids))}")

pred_img = image.copy()
for i in range(len(boxes)):
    x1, y1, x2, y2 = boxes[i].astype(int)
    cid = int(class_ids[i])
    idx = cid - 1
    color = COLORS[idx % len(COLORS)]
    name = cat_map.get(cid, f"cls{cid}")
    cv2.rectangle(pred_img, (x1, y1), (x2, y2), color, 2)
    label = f"{name} {scores[i]:.2f}"
    (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.45, 1)
    cv2.rectangle(pred_img, (x1, y1-th-6), (x1+tw, y1), color, -1)
    cv2.putText(pred_img, label, (x1, y1-4),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1)

# ── Combine side-by-side ────────────────────────────────────────
target_w = 700
scale = target_w / orig_w
target_h = int(orig_h * scale)
gt_resized = cv2.resize(gt_img, (target_w, target_h))
pred_resized = cv2.resize(pred_img, (target_w, target_h))

# Add titles
for panel, title in [(gt_resized, "GROUND TRUTH"), (pred_resized, "BASELINE PREDICTIONS")]:
    cv2.putText(panel, title, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 4)
    cv2.putText(panel, title, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,255), 2)

divider = np.full((target_h, 4, 3), 200, dtype=np.uint8)
comparison = np.hstack([gt_resized, divider, pred_resized])

# ── Build legend ────────────────────────────────────────────────
used_ids = sorted(set(a["category_id"] for a in gt_anns) | set(int(c) for c in class_ids))
legend_h = 35
legend = np.full((legend_h, comparison.shape[1], 3), 40, dtype=np.uint8)
x_off = 10
for cid in used_ids:
    idx = cid - 1
    color = COLORS[idx % len(COLORS)]
    name = cat_map.get(cid, f"cls{cid}")
    cv2.rectangle(legend, (x_off, 8), (x_off+18, 26), color, -1)
    cv2.putText(legend, name, (x_off+22, 23), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1)
    x_off += 22 + len(name)*9 + 15

final = np.vstack([comparison, legend])

# ── Display ─────────────────────────────────────────────────────
out_path = "/tmp/sanity_check_comparison.png"
cv2.imwrite(out_path, final)
display(IPImage(filename=out_path, width=1404))

# ── Category mapping verification ──────────────────────────────
print("\n" + "=" * 60)
print("  CATEGORY MAPPING VERIFICATION")
print("=" * 60)
for cid in sorted(cat_map.keys()):
    print(f"  model label {cid} -> cat_id {cid} = {cat_map[cid]}")
print("=" * 60)
print("If a car in GT is labelled 'car' in predictions (not 'van'),")
print("the category mapping is correct. Safe to run full validation!")
print("=" * 60)

---
## Step 5 — Run Validation (COCO mAP)

Runs inference on the VisDrone val set and computes:
- **mAP@50**, **mAP@50:95**
- **AP-small**, **AP-medium**, **AP-large**
- **Per-class AP@50** for all 10 VisDrone classes

Results are saved per-model with **resume support** — if Kaggle session restarts, re-run and it skips already-evaluated models.

> ⏱ ~30–60 min for all 21 models on T4

In [ ]:
#@title 5a. Run validation for ALL models
os.chdir(WORKSPACE)
!python eval/run_validation.py --config eval/config.yaml --workspace "{WORKSPACE}"

In [ ]:
#@title 5b. (Optional) Run validation for a SINGLE model (quick test)
# Uncomment one of these to test a single model first:

# os.chdir(WORKSPACE)
# !python eval/run_validation.py --config eval/config.yaml --workspace "{WORKSPACE}" --model baseline_rtdetr_r18
# !python eval/run_validation.py --config eval/config.yaml --workspace "{WORKSPACE}" --model gnconv_p2_crr
# !python eval/run_validation.py --config eval/config.yaml --workspace "{WORKSPACE}" --model efficientnet_b2_p2

---
## Step 6 — Run Latency Benchmark

Measures per-model:
- **Mean / Median / P95 / P99 latency** (ms)
- **FPS** (frames per second)
- **GPU memory** usage
- **Model size** (MB) and **parameter count**

> ⏱ ~10–20 min for all models (50 warmup + 200 measure iterations each)

In [ ]:
#@title 6a. Run benchmark for ALL models
os.chdir(WORKSPACE)
!python eval/run_benchmark.py --config eval/config.yaml --workspace "{WORKSPACE}"

In [ ]:
#@title 6b. (Optional) Also run end-to-end benchmark (includes preprocessing)
# os.chdir(WORKSPACE)
# !python eval/run_benchmark.py --config eval/config.yaml --workspace "{WORKSPACE}" --e2e

---
## Step 7 — Generate Report

Aggregates validation + benchmark results into:
- **Accuracy comparison table** (CSV)
- **Per-class AP@50 table** (CSV)
- **Speed vs Accuracy comparison** (CSV)
- **3-dec vs 6-dec comparison**
- **Plots**: mAP bar chart, Pareto scatter, AP by object size, per-class heatmap, FPS chart

In [ ]:
#@title 7. Generate full report
os.chdir(WORKSPACE)
!python eval/generate_report.py --config eval/config.yaml --workspace "{WORKSPACE}"

---
## Step 8 — View Results Inline

In [ ]:
#@title 8a. Accuracy Comparison Table
import pandas as pd
from IPython.display import display
os.chdir(WORKSPACE)

csv_path = 'eval/results/reports/accuracy_comparison.csv'
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(f"━" * 80)
    print("  ACCURACY COMPARISON")
    print(f"━" * 80)
    display(df.style.format({
        'mAP_50': '{:.4f}', 'mAP_50_95': '{:.4f}',
        'mAP_small': '{:.4f}', 'mAP_medium': '{:.4f}', 'mAP_large': '{:.4f}',
        'delta_mAP_50': '{:+.4f}'
    }).background_gradient(subset=['mAP_50'], cmap='Greens'))
else:
    print("❌ Not found. Run Steps 5 + 7 first.")

In [ ]:
#@title 8b. Speed vs Accuracy Table
csv_path = 'eval/results/reports/speed_accuracy_comparison.csv'
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(f"━" * 80)
    print("  SPEED vs ACCURACY")
    print(f"━" * 80)
    display(df.style.format({
        'mAP_50': '{:.4f}', 'mAP_50_95': '{:.4f}', 'mAP_small': '{:.4f}',
        'latency_ms': '{:.2f}', 'fps': '{:.1f}', 'size_mb': '{:.1f}',
        'efficiency_score': '{:.2f}'
    }).background_gradient(subset=['efficiency_score'], cmap='YlGn'))
else:
    print("❌ Not found. Run Steps 5 + 6 + 7 first.")

In [ ]:
#@title 8c. Per-Class AP@50
csv_path = 'eval/results/reports/per_class_ap50.csv'
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print("PER-CLASS AP@50")
    # Color each AP column
    ap_cols = [c for c in df.columns if c.startswith('AP50_')]
    display(df.style.format({c: '{:.4f}' for c in ap_cols})
            .background_gradient(subset=ap_cols, cmap='YlOrRd', vmin=0, vmax=0.7))
else:
    print("❌ Not found. Run Steps 5 + 7 first.")

In [ ]:
#@title 8d. Display All Generated Plots
from IPython.display import Image as IPImage, display
import glob

plots_dir = 'eval/results/reports/plots'
if os.path.isdir(plots_dir):
    plot_files = sorted(glob.glob(os.path.join(plots_dir, '*.png')))
    print(f"✅ Found {len(plot_files)} plots\n")
    for pf in plot_files:
        print(f"── {os.path.basename(pf)} ──")
        display(IPImage(filename=pf, width=900))
        print()
else:
    print("❌ No plots found. Run Step 7 first.")

In [ ]:
#@title 8e. Quick look at raw JSON results
import json

# Find the GPU results folder for this Kaggle session
results_root = 'eval/results'
gpu_dirs = [d for d in os.listdir(results_root)
            if os.path.isdir(os.path.join(results_root, d)) and d.startswith('GPU_')]
print(f"GPU result folders: {gpu_dirs}\n")

for gpu_dir in gpu_dirs:
    # Validation summary
    val_file = os.path.join(results_root, gpu_dir, 'validation', 'all_validation_results.json')
    if os.path.exists(val_file):
        with open(val_file) as f:
            val = json.load(f)
        print(f"━ {gpu_dir} — Validation: {len(val)} models evaluated")
        for k, v in val.items():
            m = v['metrics']
            print(f"  {v['name']:<40} mAP@50={m['mAP_50']:.4f}  mAP@50:95={m['mAP_50_95']:.4f}  AP-S={m['mAP_small']:.4f}")

    # Benchmark summary
    bench_file = os.path.join(results_root, gpu_dir, 'benchmark', 'benchmark_results.json')
    if os.path.exists(bench_file):
        with open(bench_file) as f:
            bench = json.load(f)
        print(f"\n━ {gpu_dir} — Benchmark: {len(bench)} models benchmarked")
        for k, v in bench.items():
            print(f"  {v['name']:<40} {v['mean_latency_ms']:.2f}ms  FPS={v['fps']:.1f}  P95={v['p95_latency_ms']:.2f}ms")

---
## Step 9 — Save & Download Results

In [ ]:
#@title 9a. Zip results for download
import shutil
os.chdir(WORKSPACE)

results_src = os.path.join(WORKSPACE, 'eval/results')
zip_path = '/kaggle/working/eval_results'

if os.path.exists(results_src):
    shutil.make_archive(zip_path, 'zip', results_src)
    size_mb = os.path.getsize(zip_path + '.zip') / (1024 * 1024)
    print(f"✅ Zipped: {zip_path}.zip ({size_mb:.1f} MB)")
    print(f"\n📥 Download from the Output tab on the right panel →")
    print(f"   Or find it at: /kaggle/working/eval_results.zip")
else:
    print("❌ No results to download yet.")

In [ ]:
#@title 9b. Save results as Kaggle output dataset
# Everything in /kaggle/working/ is automatically saved as notebook output.
# Copy results to the working directory root so they appear in Output.
import shutil

output_dst = '/kaggle/working/eval_results_output'
results_src = os.path.join(WORKSPACE, 'eval/results')

if os.path.exists(results_src):
    if os.path.exists(output_dst):
        shutil.rmtree(output_dst)
    shutil.copytree(results_src, output_dst)
    print(f"✅ Results copied to {output_dst}")
    print("   These will be available in the notebook Output after 'Save & Run All'.")
else:
    print("❌ No results to copy yet.")

In [ ]:
#@title 9c. (Optional) Git commit & push new Kaggle results back to repo
# This pushes the Kaggle GPU results alongside your existing RTX 4050 results.
# You'll need a GitHub Personal Access Token.

# os.chdir(WORKSPACE)
# !git config user.email "your-email@example.com"
# !git config user.name "Your Name"
# !git add eval/results/
# !git commit -m "Add Kaggle T4 GPU evaluation results"
# !git push origin main
# # For private repos: git push https://<TOKEN>@github.com/Sasankamadura/FYP-Final-Testing.git main

---
## 💡 Tips & Troubleshooting

### Resume Support
The validation script **automatically skips** models that have already been evaluated.
If your Kaggle session dies mid-run, just re-run Step 5 — it picks up where it left off.

### Kaggle Dataset Setup
| Step | Details |
|------|---------|
| **Upload checkpoints** | kaggle.com → Datasets → New Dataset → upload the two `Checkpoints` folders |
| **Upload VisDrone data** | Same process for `VisDrone Val image set` and `VisDrone test image set` |
| **Add to notebook** | Click 'Add Data' in the right panel → search for your datasets |
| **Check paths** | Look at `/kaggle/input/` to see exact folder names |

### Cross-GPU Comparison
Your repo already has results from `GPU_NVIDIA_GeForce_RTX_4050_Laptop_GPU`.
After running on Kaggle (T4), the report generator will automatically produce a
**cross-GPU FPS comparison** table.

### Performance Tips
| Tip | Details |
|-----|---------|
| **Kaggle GPU quota** | Free tier: 30 hrs/week GPU. Use wisely! |
| **T4 x2** | Select T4 x2 accelerator for best compatibility |
| **Single model test** | Use `--model baseline_rtdetr_r18` flag to test one model quickly |
| **E2E benchmark** | Add `--e2e` flag to also measure full pipeline |
| **Session timeout** | Kaggle sessions timeout after ~12 hrs. Save results periodically |

### Common Issues
| Issue | Fix |
|-------|-----|
| `CUDAExecutionProvider not available` | Settings → Accelerator → GPU T4 x2 |
| `FileNotFoundError` on `.onnx` file | Check symlinks in Step 3b — dataset paths must match |
| `Dataset not found` | Click 'Add Data' button → search and add your uploaded datasets |
| `Session crashed` during validation | Re-run Step 5 (resume support will skip completed models) |
| `Read-only filesystem` error | Kaggle input is read-only; results write to `/kaggle/working/` |
| `git clone` fails (private repo) | Use `!git clone https://<TOKEN>@github.com/...` |